In [93]:
import cv2
import pandas as pd
import numpy as np
import json
import pybboxes as pbx

In [94]:
pd.options.mode.chained_assignment = None 
HEIGHT = 720
WIDTH = 1280

In [96]:
def preprocess_and_save(trim_data):
    for i, trim in enumerate(trim_data, start=1):
        project = list(trim['projects.clgwbo68i04k1071t3d8i4qo1.labels'][0]['annotations']['frames'].items())
        matrix = []
        for frame in project:
            frame_num = frame[0]
            objects = frame[1]['objects']
            obj_id = objects.keys()

            for aidi in obj_id:
                row = []
                bbox = [k[1] for k in [*objects[aidi]['bounding_box'].items()]]
                row.append([frame_num, aidi, objects[aidi]['name'], *bbox])
                matrix.extend(row)

        df = pd.DataFrame(matrix, columns=['frame', 'object_id', 'label', 'top', 'left', 'height', 'width'])
        df.frame = df.frame.astype(int)

        monster_df = pd.DataFrame()
        for obj in df.object_id.unique().tolist():
            over_df = pd.DataFrame()
            slice_bob = df[df.object_id == obj].sort_values('frame')
            for first, second in zip(slice_bob.frame.unique().tolist(), slice_bob.frame.unique().tolist()[1:]):
                frame_numbers = range(first, second + 1)
                sub_set_first = slice_bob[slice_bob.frame == first][['top', 'left', 'height', 'width']]
                sub_set_second = slice_bob[slice_bob.frame == second][['top', 'left', 'height', 'width']]
                temp_df = pd.DataFrame({'frame': frame_numbers, 
                                        'object_id': obj, 
                                        'label': slice_bob.label.iloc[0], 
                                        'top': sub_set_first.top.iloc[0], 
                                        'left': sub_set_first.left.iloc[0], 
                                        'height': sub_set_first.height.iloc[0], 
                                        'width': sub_set_first.width.iloc[0]})
                temp_df['top'].iloc[1:] = None
                temp_df['top'].iloc[-1] = sub_set_second.top.iloc[0]
                temp_df['left'].iloc[1:] = None
                temp_df['left'].iloc[-1] = sub_set_second.left.iloc[0]
                temp_df['height'].iloc[1:] = None
                temp_df['height'].iloc[-1] = sub_set_second.height.iloc[0]
                temp_df['width'].iloc[1:] = None
                temp_df['width'].iloc[-1] = sub_set_second.width.iloc[0]
                temp_df = temp_df.interpolate()
                over_df = pd.concat([over_df, temp_df])
            monster_df = pd.concat([monster_df, over_df])

        monster_df = monster_df.drop_duplicates().sort_values('frame')
        monster_df = monster_df[['frame', 'object_id', 'label', 'top', 'left', 'width', 'height']]

        monster_df['x_center'] = (monster_df['left'] + monster_df['width'] / 2) / WIDTH
        monster_df['y_center'] = (monster_df['top'] + monster_df['height'] / 2) / HEIGHT
        monster_df['width_norm'] = monster_df['width'] / WIDTH
        monster_df['height_norm'] = monster_df['height'] / HEIGHT
        monster_df = monster_df[['frame', 'label', 'x_center', 'y_center', 'width_norm', 'height_norm']]

        vidcap = cv2.VideoCapture(f'Video/Локотех_Trim{i}.mp4')
        success, image = vidcap.read()
        count = 1

        while success:
            cv2.imwrite(f"Images/{i}_{count}.jpg", image)
            annotations = monster_df[monster_df.frame == count].drop(columns='frame')
            annotations.to_csv(f"Images/{i}_{count}.txt", header=None, index=None, sep=' ')

            success,image = vidcap.read()
            count += 1

In [86]:
with open('bboxes.json', 'r') as j:
    contents = json.loads(j.read())
    preprocess_and_save(contents)
    contents.close()

In [13]:
import glob
import os
import random 

def train_val_split(folder):
    if os.getcwd() != folder:
        os.chdir(f"{folder}")
    
    images = glob.glob("*.jpg")
    annots = glob.glob("*.txt")
    
    val_images = random.sample(images, int(len(images) * 0.2))
    test_images = random.sample(val_images, int(len(val_images) * 0.3))
    
    train_folder = '/Users/dushamishkin/Desktop/Jupyter/HAR/yolov7/train'
    test_folder = '/Users/dushamishkin/Desktop/Jupyter/HAR/yolov7/test'
    val_folder = '/Users/dushamishkin/Desktop/Jupyter/HAR/yolov7/valid'
    
    for name in images:
        name = name[:-4]
        if f"{name}.jpg" in val_images:
            
            if f"{name}.jpg" in test_images:
                os.replace(f"{folder}/{name}.jpg", f"{test_folder}/images/{name}.jpg")
                os.replace(f"{folder}/{name}.txt", f"{test_folder}/labels/{name}.txt")
            
            else:
                os.replace(f"{folder}/{name}.jpg", f"{val_folder}/images/{name}.jpg")
                os.replace(f"{folder}/{name}.txt", f"{val_folder}/labels/{name}.txt")
            
        else:
            os.replace(f"{folder}/{name}.jpg", f"{train_folder}/images/{name}.jpg")
            os.replace(f"{folder}/{name}.txt", f"{train_folder}/labels/{name}.txt")
        
    os.chdir('/Users/dushamishkin/Desktop/Jupyter/HAR/')

In [14]:
train_val_split('/Users/dushamishkin/Desktop/Jupyter/HAR/Images')

In [ ]:
# DO NOT RUN CODE BELOW

In [32]:
def show_frames_w_bboxes(trim_name, df):
    vidcap = cv2.VideoCapture(f'Video/Локотех_{trim_name}.mp4')
    success, image = vidcap.read()
    count = 1
        
    while success:
        objects = df[df.frame == count]
        for _, obj in objects.iterrows():
            name = obj.label
        
            if name == 'bot':
                color = (0, 230, 255)
            elif name == 'mob':
                color = (188, 0, 255)
            else:
                color = (255, 255, 255)
                
            t = int(obj.top)
            h = int(obj.height)
            l = int(obj.left)
            w = int(obj.width)
                
            cv2.rectangle(image, (l + w, t), (l, t + h), color, 2)
            
        cv2.imshow('lol', image)
        cv2.waitKey(0)
        cv2.destroyWindow('lol')
        success, image = vidcap.read()
        count += 1

In [35]:
show_frames_w_bboxes('Trim3', monster_df)

error: OpenCV(4.7.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window_w32.cpp:1261: error: (-27:Null pointer) NULL window: 'lol' in function 'cvDestroyWindow'
